In [ ]:
!pip install jieba
!pip install snownlp

In [ ]:
#### 對愛食記評論和食記內容進行文本分析(詞頻) ####     =>  由於資料意義不大,故僅提供參考
import pandas as pd
import json 
import time
import jieba

### 編寫詞頻分析函式
def Jieba_Analysis(company,restaurant):
    ## 將CSV內的分店留言內容與食記內容存成一整個字串
    try:    # 開到沒資料的csv會報錯
        df = pd.read_csv(f'C:/Users/user/Desktop/ifoodie/{company}/{restaurant}.csv')    # 路徑請視自身環境更動
        for row in range(df.shape[0]):
            # 列出每一列的評論內容
            review_content = str(df.iloc[row]['文章內容'])

            # 整合每一列的食記內容
            article_content = ""
            content_segment = df.iloc[row]['食記資訊'].split("'內容':")
            time_segment = df.iloc[row]['食記資訊'].split("'發布時間': '")
            for L in range(len(content_segment)-1):
                content_row = content_segment[L+1].split("}, {'文章")[0]
                article_time = time_segment[L+1].split("', '內容'")[0].strip()
                article_timestamp = int(time.mktime(time.strptime(article_time,"%Y/%m/%d")))
                Stopby_timestamp = int(time.mktime(time.strptime("2021/4/1","%Y/%m/%d")))

                # 只對一年內的文章進行分析
                if article_timestamp > Stopby_timestamp :
                    article_content += content_row

            # 將兩者合併成一個字串
            all_article_string = review_content + article_content

            ## 進行詞頻分析
            word_count = {}
            s_list = jieba.cut(all_article_string)
            for i in s_list:
                if i in word_count:
                    word_count[i] += 1
                else:
                    word_count[i] = 1

            sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
            print(sorted_word_count)

    except:
        print ("No article available!")


### 執行看看
Jieba_Analysis("王品集團","夏慕尼")

In [ ]:
#### 對愛食記評論和食記內容進行文本分析(情感) ####
import pandas as pd
import json 
import time
from snownlp import SnowNLP
from snownlp import sentiment

### 編寫文本分析函式
def Emotion_Analysis(company,restaurant):
    ## 將CSV內的分店留言內容與食記內容存成一整個字串
    try:    # 開到沒資料的csv會報錯
        df = pd.read_csv(f'C:/Users/user/Desktop/ifoodie/{company}/{restaurant}.csv')   # 路徑請視自身環境更動
        df2 = df.copy()
        good_count_list= []
        avg_score_list = []
        bad_count_list= []

        for row in range(df.shape[0]):
            # 列出每一列的評論內容
            review_content = str(df.iloc[row]['文章內容'])

            # 整合每一列的食記內容
            article_content = ""
            content_segment = df.iloc[row]['食記資訊'].split("'內容':")
            time_segment = df.iloc[row]['食記資訊'].split("'發布時間': '")
            for L in range(len(content_segment)-1):
                content_row = content_segment[L+1].split("}, {'文章")[0]
                article_time = time_segment[L+1].split("', '內容'")[0].strip()
                article_timestamp = int(time.mktime(time.strptime(article_time,"%Y/%m/%d")))
                Stopby_timestamp = int(time.mktime(time.strptime("2021/4/1","%Y/%m/%d")))

                # 只對一年內的文章進行分析
                if article_timestamp > Stopby_timestamp :
                    article_content += content_row

            # 將兩者合併成一個字串
            all_article_string = review_content + article_content

            ## 進行情感分析
            total_score = 0
            good_count = 0
            bad_count = 0

            s = SnowNLP(all_article_string) 
            for sentence in s.sentences : 
                if "nan" in sentence:
                    pass
                else:
                    sentiments_score = SnowNLP(sentence).sentiments 
                    if sentiments_score > 0.5:
                        good_count += 1
                    elif sentiments_score <= 0.5:
                        bad_count += 1
                    total_score += sentiments_score

            avg_score = total_score/len(s.sentences)
            avg_score_list.append(avg_score)
            good_count_list.append(good_count)
            bad_count_list.append(bad_count)

        # 將評分結果新增至原df並建新的CSV(路徑請視自身環境更動)
        df2["正評數"] = good_count_list
        df2["負評數"] = bad_count_list
        df2["情感平均分數"] = avg_score_list
        df2.to_csv(f"C:\\Users\\user\\Desktop\\ifoodie2\\{company}\\{restaurant}2.csv",index=0,encoding='utf-8-sig',header=['來源網站','集團','品牌','標題','連結','菜系','評分','留言數','文章內容','食記資訊',"正評數","負評數","情感平均分數"])

    except:
        pass

### 為各個集團創建資料夾存放新的CSV檔(路徑請視自身環境更動)
restaurant_list = [{"company":"王品集團","restaurant": ["王品牛排","西堤","陶板屋","原燒","聚北海道鍋物","藝奇","夏慕尼","品田牧場","石二鍋","hot 7","莆田","青花驕","享鴨","丰禾","樂越","12MINI","THE WANG","和牛涮","尬鍋","肉次方"]},
                   {"company":"瓦城泰統","restaurant": ['瓦城','非常泰','1010湘','大心','時時香','月月THAI','YABI KITCHEN']},
                   {"company":"漢來美食","restaurant": ['漢來海港','漢來名人坊','漢來蔬食','漢來軒','翠園粵菜','五梅先生','安那居','上菜','福園台菜','弁慶','精緻海鮮火鍋','糕餅小舖','大廳酒廊','漢來咖啡廳','Pavo','焰·鐵板燒']}]

for n in range(3):
  if not os.path.exists(f"C:\\Users\\user\\Desktop\\ifoodie2\\{restaurant_list[n]['company']}"):
    os.mkdir(f"C:\\Users\\user\\Desktop\\ifoodie2\\{restaurant_list[n]['company']}")

  for restaurant in range(len(restaurant_list[n]["restaurant"])):
    Emotion_Analysis(restaurant_list[n]["company"],restaurant_list[n]["restaurant"][restaurant])   

In [ ]:
#### 對Tripadvisor.csv評論內容進行文本分析(情感) ####
#### p.s : 因Tripadvisor文章較少,故不另限制文章發布日期
import pandas as pd
import json 
import time
from snownlp import SnowNLP
from snownlp import sentiment

### 編寫文本分析函式
def Emotion_Analysis(filename):
    df = pd.read_csv(f'C:/Users/SCE/Desktop/{filename}.csv')   # 路徑請視自身環境更動
    df2 = df.copy()
    good_count_list= []
    avg_score_list = []
    bad_count_list= []

    for row in range(df.shape[0]):
        ## 列出每一列的評論內容,進行情感分析
        review_content = str(df.iloc[row]['文章內容'])
        total_score = 0
        good_count = 0
        bad_count = 0

        s = SnowNLP(review_content)
        for sentence in s.sentences : 
            if "nan" in sentence:
                pass
            else:
                sentiments_score = SnowNLP(sentence).sentiments 
                total_score += sentiments_score
                if sentiments_score > 0.5:
                    good_count += 1
                elif sentiments_score <= 0.5:
                    bad_count += 1           
                
        avg_score = total_score/len(s.sentences)
        avg_score_list.append(avg_score)
        good_count_list.append(good_count)
        bad_count_list.append(bad_count)

    ## 將評分結果新增至原df並建新的CSV(路徑請視自身環境更動)
    df2["正評數"] = good_count_list
    df2["負評數"] = bad_count_list
    df2["情感平均分數"] = avg_score_list
    df2.to_csv(f"C:\\Users\\SCE\\Desktop\\{filename}2.csv",index=0,encoding='utf-8-sig',header=['來源網站','集團','品牌','標題','店家地址','評論標題','發布時間','文章內容','評分',"Lat","Lon","正評數","負評數","情感平均分數"])

### 執行
Emotion_Analysis("Tripadvisor")

In [1]:
#### 對google_map.csv評論內容進行文本分析(情感) ####
import pandas as pd
import json 
import time
from snownlp import SnowNLP
from snownlp import sentiment

### 編寫文本分析函式
def Emotion_Analysis(filename):
    df = pd.read_csv(f'C:/Users/SCE/Desktop/{filename}.csv')   # 路徑請視自身環境更動
    df2 = df.copy()
    good_count_list= []
    avg_score_list = []
    bad_count_list= []

    for row in range(df.shape[0]):
        try:
            # 列出每一列的評論內容及時間
            review_content = str(df.iloc[row]['文章內容'])
            article_time = df.iloc[row]['發布時間']
            article_timestamp = int(time.mktime(time.strptime(article_time,"%Y/%m/%d")))
            Stopby_timestamp = int(time.mktime(time.strptime("2021/4/1","%Y/%m/%d")))
        except:
            pass

        ## 進行情感分析
        total_score = 0
        good_count = 0
        bad_count = 0
        
        # 只對一年內的文章進行分析
        if article_timestamp > Stopby_timestamp :
            s = SnowNLP(review_content)
            for sentence in s.sentences : 
                if "nan" in sentence:
                    pass
                else:
                    sentiments_score = SnowNLP(sentence).sentiments 
                    total_score += sentiments_score
                    if sentiments_score > 0.5:
                        good_count += 1
                    elif sentiments_score <= 0.5:
                        bad_count += 1           
                
        avg_score = total_score/len(s.sentences)
        avg_score_list.append(avg_score)
        good_count_list.append(good_count)
        bad_count_list.append(bad_count)

    ## 將評分結果新增至原df並建新的CSV(路徑請視自身環境更動)
    df2["正評數"] = good_count_list
    df2["負評數"] = bad_count_list
    df2["情感平均分數"] = avg_score_list
    df2.to_csv(f"C:\\Users\\SCE\\Desktop\\{filename}2.csv",index=0,encoding='utf-8-sig',header=['來源網站','集團','品牌','標題','地址','發布時間','文章內容','評分',"正評數","負評數","情感平均分數"])

### 執行
Emotion_Analysis("google_map")

In [ ]:
#### 對痞客邦.csv評論內容進行文本分析(情感) ####
import pandas as pd
import json 
import time
from snownlp import SnowNLP
from snownlp import sentiment

### 編寫文本分析函式
def Emotion_Analysis(filename):
    df = pd.read_csv(f'C:/Users/SCE/Desktop/{filename}.csv')   # 路徑請視自身環境更動
    df2 = df.copy()
    good_count_list= []
    avg_score_list = []
    bad_count_list= []

    for row in range(df.shape[0]):
        try:
            # 列出每一列的評論內容及時間
            review_content = str(df.iloc[row]['文章內容'])
            article_time = df.iloc[row]['發布時間']
            article_timestamp = int(time.mktime(time.strptime(article_time,"%Y/%m/%d")))
            Stopby_timestamp = int(time.mktime(time.strptime("2021/4/1","%Y/%m/%d")))
        except:
            pass

        ## 進行情感分析
        total_score = 0
        good_count = 0
        bad_count = 0

        # 只對一年內的文章進行分析
        if article_timestamp > Stopby_timestamp :
            s = SnowNLP(review_content)
            for sentence in s.sentences : 
                if "nan" in sentence:
                    pass
                else:
                    sentiments_score = SnowNLP(sentence).sentiments 
                    total_score += sentiments_score
                    if sentiments_score > 0.5:
                        good_count += 1
                    elif sentiments_score <= 0.5:
                        bad_count += 1           
                
        avg_score = total_score/len(s.sentences)
        avg_score_list.append(avg_score)
        good_count_list.append(good_count)
        bad_count_list.append(bad_count)

    ## 將評分結果新增至原df並建新的CSV(路徑請視自身環境更動)
    df2["正評數"] = good_count_list
    df2["負評數"] = bad_count_list
    df2["情感平均分數"] = avg_score_list
    df2.to_csv(f"C:\\Users\\SCE\\Desktop\\{filename}2.csv",index=0,encoding='utf-8-sig',header=['來源網站','集團','品牌','發布時間','標題','文章內容',"瀏覽數","留言數","正評數","負評數","情感平均分數"])

### 執行
Emotion_Analysis("PIXNET")